In [27]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### 1단계: 문서 분할 설정

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

### 2단계: 임베딩 모델 설정

In [29]:
from langchain_openai import OpenAIEmbeddings
import numpy as np

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536,                 #  고성능 임베딩 차원
    openai_api_key=OPENAI_API_KEY
)

### 3단계: 검색기 설정

In [30]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 경로 (실제 경로로 수정 필요)
pdf_path = "../data/콘텐츠분쟁해결_사례.pdf"

print("📄 PDF 문서 로드 중...")

try:
    # PDF 파일 로드
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print(f"✅ 로드된 문서 페이지 수: {len(documents)}")
    
    # 문서 분할
    split_docs = text_splitter.split_documents(documents)
    print(f"✅ 분할된 청크 수: {len(split_docs)}")
    
    # 벡터스토어 생성
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    print("✅ 벡터스토어 생성 완료!")
    
    # 검색기 설정 (이 부분이 중요!)
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )
    print("✅ 검색기 설정 완료!")
    
except Exception as e:
    print(f"❌ PDF 로드 및 벡터스토어 생성 오류: {e}")
    print("PDF 파일 경로를 확인해주세요.")
    # 오류 발생시 시스템 종료
    raise

📄 PDF 문서 로드 중...
✅ 로드된 문서 페이지 수: 109
✅ 분할된 청크 수: 104
✅ 벡터스토어 생성 완료!
✅ 검색기 설정 완료!


### 4단계: LLM 설정 

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",                 # 또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                # 법률 조언은 정확성 중시
    max_tokens=2000,                # 충분한 답변 길이
    openai_api_key=OPENAI_API_KEY
)

### 5단계: 법률 자문 프롬프트 작성

In [32]:
from langchain.prompts import PromptTemplate

prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

### 6단계: QA 체인 생성 

In [33]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                        # 위에서 설정한 LLM
    chain_type="stuff",             # 문서들을 하나로 합쳐서 처리
    retriever=retriever,            # 위에서 설정한 검색기
    chain_type_kwargs={"prompt": prompt},  # 법률 자문 프롬프트
    return_source_documents=True    # 참조 문서도 함께 반환
)

### 7단계: 테스트 질문 작성 

In [34]:
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

### 8단계: 분쟁 유형 분류 함수 ( 선택 )

In [35]:
def classify_dispute_type(query):
    """질문 내용을 분석하여 분쟁 유형을 분류하는 함수"""
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

### 9단계: 법률 자문 실행 함수

In [36]:
def get_legal_advice(question):
    """질문에 대한 법률 자문을 제공하는 함수"""
    try:
        # 분쟁 유형 분류
        dispute_type = classify_dispute_type(question)
        print(f"분쟁 유형: {dispute_type}")
        print("-" * 50)
        
        # RAG 체인 실행
        result = qa_chain({"query": question})
        
        # 결과 출력
        print("📋 법률 자문 결과:")
        print(result["result"])
        print("\n" + "="*50)
        
        # 참조 문서 정보 출력
        if result.get("source_documents"):
            print("📚 참조된 사례 문서:")
            for i, doc in enumerate(result["source_documents"][:3]):  # 상위 3개만 표시
                print(f"[참조 {i+1}] 페이지: {doc.metadata.get('page', 'N/A')}")
                print(f"내용 미리보기: {doc.page_content[:200]}...")
                print("-" * 30)
        
        return result
        
    except Exception as e:
        print(f"법률 자문 처리 중 오류 발생: {e}")
        return None

### 10단계: 테스트 실행

In [37]:
def run_tests():
    """테스트 질문들을 실행하는 함수"""
    for i, question in enumerate(test_questions, 1):
        print(f"\n🔍 테스트 질문 {i}:")
        print(f"질문: {question}")
        print("="*70)
        
        result = get_legal_advice(question)
        
        if result:
            print("\n✅ 자문 완료")
        else:
            print("\n❌ 자문 실패")
        
        print("\n" + "🔄"*70 + "\n")

### 11단계: 대화형 자문 시스템

In [38]:
def interactive_consultation():
    """대화형 법률 자문 시스템"""
    print("💼 콘텐츠 분쟁 해결 법률 자문 시스템")
    print("=" * 50)
    print("질문을 입력하세요 (종료하려면 'quit' 입력)")
    
    while True:
        user_question = input("\n🤔 질문: ").strip()
        
        if user_question.lower() in ['quit', 'exit', '종료', '나가기']:
            print("👋 법률 자문을 종료합니다.")
            break
        
        if not user_question:
            print("질문을 입력해주세요.")
            continue
        
        print("\n🔍 분석 중...")
        result = get_legal_advice(user_question)
        
        if not result:
            print("죄송합니다. 처리 중 오류가 발생했습니다.")

### 메인 실행부

In [ ]:
if __name__ == "__main__":
    # 시스템 준비 확인
    if 'vectorstore' in locals() and vectorstore is not None:
        print("✅ RAG 시스템 준비 완료!")
        
        # 사용자 선택
        print("\n실행 옵션을 선택하세요:")
        print("1. 테스트 질문 실행")
        print("2. 대화형 자문 시작")
        print("3. 단일 질문 테스트")
        
        choice = input("\n선택 (1/2/3): ").strip()
        
        if choice == "1":
            run_tests()
        elif choice == "2":
            interactive_consultation()
        elif choice == "3":
            test_question = "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?"
            print(f"\n테스트 질문: {test_question}")
            get_legal_advice(test_question)
        else:
            print("올바른 옵션을 선택해주세요.")
    else:
        print("❌ RAG 시스템 초기화에 실패했습니다.")
        print("PDF 파일 경로와 환경변수를 확인해주세요.")

✅ RAG 시스템 준비 완료!

실행 옵션을 선택하세요:
1. 테스트 질문 실행
2. 대화형 자문 시작
3. 단일 질문 테스트

🔍 테스트 질문 1:
질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
분쟁 유형: 게임
--------------------------------------------------


C:\Users\wlwog\AppData\Local\Temp\ipykernel_16616\508184463.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


📋 법률 자문 결과:
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다. 제시된 사례들을 근거로 답변을 구성하겠습니다.

### 1. 사례 분석

**사례 1 (2006년):**  
- **사건개요:** 시스템 오류로 아이템이 소멸되었으나, 계정 명의자가 아니어서 복구가 거부됨.
- **처리결과:** 계정 명의자가 복구 요청을 해야 한다고 안내.

**사례 2 (2009년):**  
- **사건개요:** 시스템 오류로 게임머니가 소실되었다고 주장했으나, 시스템 오류가 확인되지 않음.
- **처리결과:** 통신회선의 문제일 가능성을 설명하고, 시스템 오류가 없음을 안내.

**사례 3 (2009년):**  
- **사건개요:** 해킹으로 아이템이 손실되었으나, 보안서비스 미사용으로 복구 거부.
- **처리결과:** 보안서비스 미사용으로 인해 복구 불가.

**사례 4 (2007년):**  
- **사건개요:** 해킹 아이템을 구매하여 회수되었고, 게임머니는 환급됨.
- **처리결과:** 아이템 회수는 정당하며, 게임머니 환급으로 피해가 없다고 판단.

**사례 5 (2008년):**  
- **사건개요:** 보안서비스 미작동으로 아이템 도난.
- **처리결과:** 보안서비스 미작동이 확인되어 아이템 복구 권고.

### 2. 관련 법령 및 조항

- **민법 제250조 및 제251조:** 도품이나 유실물에 대한 반환 청구에 관한 규정.
- **게임사의 약관 및 운영정책:** 계정 명의자 문제, 보안서비스 사용 여부 등이 중요.

### 3. 실무적 해결방안

1. **계정 명의 확인:**
   - 계정 명의자가 본인이 아니라면, 명의자가 직접 복구 요청을 해야 합니다. 계정 명의 문제는 게임사의 약관에 따라 처리됩니다.

2. **시스템 오류 증명:**
   - 시스템 오류로 인한 손실을 주장하려면, 오류 발생을 입증할 수 있는 증거가 필요합니다. 다른 사용자들에게 유사한 문제가 발생했는지